# Ajuste de un histograma paso a paso

## Datos

In [ ]:
import pandas as pd
data = pd.read_csv("histogram.csv")
xdata = data["x"]
ydata = data["y"]
print(data)

## Modelo

El modelo es una PDF normal escalada por una constante de normalización.  
Parámetros:
- Constante de normalización
- Media $\mu$
- Desviación estándar $\sigma$

In [ ]:
from scipy.stats import norm
def fit_model(x, par):
    return par[0] * norm.pdf(x, loc=par[1], scale=par[2])

## Función de costo de un dato

Función de costo para variable dependiente y que sigue una distribución de Poisson

In [ ]:
import math
def poisson_cost(mu, y):
    if y==0:
        cost = 2 * mu
    else:
        cost = 2 * (mu - y) - 2 * y * math.log(mu/y)  
    return cost

## Función de costo del ajuste

In [ ]:
def fit_cost(x, y, par):     
    mu = fit_model(x, par)
    cost = 0   
    for mu1, y1 in zip(mu, y):
        cost += poisson_cost(mu1, y1)
    return cost

In [ ]:
J = lambda par: fit_cost(xdata, ydata, par)

## Minimización

In [ ]:
import numpy as np
from scipy.optimize import minimize
initial_parameters = np.array([1, 0, 1])
res = minimize(J, x0=initial_parameters)
res

## Parámetros

In [ ]:
par_est = res.x
cova = 2*res.hess_inv
error = np.sqrt( np.diagonal(cova) )
rho = cova[0][1]/(error[0]*error[1])
print(f"Normalization = {par_est[0]:.2f} ± {error[0]:.2f}")
print(f"Mean = {par_est[1]:.3f} ± {error[1]:.3f}")
print(f"Standard deviation = {par_est[2]:.3f} ± {error[2]:.3f}")

## Bondad del ajuste

In [ ]:
from scipy.stats import chi2
cost_min = res.fun
ndof = len(xdata) - len(par_est)
pvalor = chi2.sf(cost_min, ndof)
print(f'χ²min = {cost_min:.2f}')
print(f'ndof = {ndof}')
print(f'pvalor = {pvalor*100:.1f}%')

## Plot del ajuste

In [ ]:
xfit = np.linspace( xdata.min(), xdata.max(), 256)
mu_est = fit_model(xfit, par_est)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.set_xlabel("x")
ax.set_ylabel("Events")
yerror = np.sqrt(ydata)
ax.errorbar(xdata, ydata, yerror,ls='none', marker='o', ms=4, label="Data")
ax.plot(xfit, mu_est, label="Fit")
plt.legend()
plt.show()